In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy import stats
import researchpy as rp

from statsmodels.formula.api import ols

In [3]:
bike_sharing_data = pd.read_csv('datasets/bike_sharing_data_processed.csv')
bike_sharing_data.head()

,season,mnth,holiday,workingday,weathersit,temp,cnt
0,1,1,0,0,2,0.344167,985
1,1,1,0,0,2,0.363478,801
2,1,1,0,1,1,0.196364,1349
3,1,1,0,1,1,0.200000,1562
4,1,1,0,1,1,0.226957,1600


In [4]:
bike_sharing_data.shape

(731, 7)

In [5]:
bike_sharing_data['weathersit'].unique()

array([2, 1, 3])

In [8]:
bike_sharing_data.groupby('weathersit')['cnt'].describe()

,count,mean,std,min,25%,50%,75%,max
weathersit,,,,,,,,
1,463.0,4876.786177,1879.483989,431.0,3636.5,4844.0,6458.5,8714.0
2,247.0,4035.862348,1809.109918,605.0,2674.5,4040.0,5257.5,8362.0
3,21.0,1803.285714,1240.284449,22.0,705.0,1817.0,2424.0,4639.0


**Categories**

*1 - Sunny day* <br>
*2 - Little more clouds*<br>
*3 - Cloudy with thunderstorms*

<h4> Hypothesis</h4>

-  Null hypothesis (H0): The means of these are the same 
-  Alternative hypothesis (H1): The means of these are different

bike_sharing_data.boxplot(column=['cnt'], by='weathersit', figsize=(12, 8))

More bikes are rented during sunny day than on cloudy and thunderstorm days

<h4> Calculating one way ANOVA </h4>

In [10]:
stats.f_oneway(bike_sharing_data['cnt'][bike_sharing_data['weathersit'] == 1],
              bike_sharing_data['cnt'][bike_sharing_data['weathersit'] == 2],
              bike_sharing_data['cnt'][bike_sharing_data['weathersit'] == 3])

F_onewayResult(statistic=40.06604492024684, pvalue=3.106317270053755e-17)

 The p-value returned is very very tiny and far below 5% and this indicates the differences in the means between these 3 categories/samples are significant. Therefore, the weather situtation has an impact on the count of the number of bikes rented.

However, this test does not show which specific group's means compared with each other are different, i.e, group 1, 2 are different, or group 2, 3 are different or group 1, 3 are different. That's why we use another test, **Tukey's Honest Significance Difference test**

<h3>Tukey's Honest Significance Difference Test</h3>

In [13]:
from statsmodels.stats.multicomp import MultiComparison
mul_com = MultiComparison(bike_sharing_data['cnt'], bike_sharing_data['weathersit'])

mul_result = mul_com.tukeyhsd()

print(mul_result)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj   lower      upper    reject
-----------------------------------------------------------
     1      2  -840.9238 0.001  -1181.639  -500.2086   True
     1      3 -3073.5005 0.001 -4038.2706 -2108.7304   True
     2      3 -2232.5766 0.001 -3215.4794 -1249.6738   True
-----------------------------------------------------------


The above table consists of the comparison between all three groups. On the last column, the values are for the comparison of all three groups, 'True' which means we reject null hypothesis (H0) and accept the alternative hypothesis.